# # Chord Progressions Notebook
Este Notebook demuestra un **pipeline** end-to-end:

1. Generar progresiones en **.mid** usando la lógica de JazzNet (`generate_progression.py`)
2. Convertir los `.mid` a **.wav** usando Timidity (`audio_conversion.py`)
3. Crear archivos **.jams** con etiquetas de acordes (compatibles con el regex de JAMS) y numerales romanos en el `sandbox` (`jams_creation.py` + `roman_to_chord.py`).

---

## 1. Configuración de Entorno
Asegúrate de que:
- El directorio raíz contenga la carpeta `src/`.
- Tienes instalado Timidity y las dependencias (ver `requirements.txt`).


In [1]:
import sys
import os
from pathlib import Path

# Ajustar la ruta para acceder al paquete src/
PROJECT_ROOT = Path.cwd().parent  # Ajustar si es necesario
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config import MIDI_DIR, WAV_DIR, JAMS_DIR, DEFAULT_TEMPO
from src.generate_progression import generate_progression
from src.audio_conversion import convert_all_mid_in_folder
from src.jams_creation import create_jams_for_folder

print("MIDI_DIR:", MIDI_DIR)
print("WAV_DIR:", WAV_DIR)
print("JAMS_DIR:", JAMS_DIR)
print("DEFAULT_TEMPO:", DEFAULT_TEMPO)


MIDI_DIR: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi
WAV_DIR: /home/cepatinog/MIR/final_project/chord_progression_generator/data/wav
JAMS_DIR: /home/cepatinog/MIR/final_project/chord_progression_generator/data/jams
DEFAULT_TEMPO: 60


## 2. Generar Progresiones .mid

Veremos 2 ejemplos:
1. Progresión de 3 acordes: **`ii,7 - V,7 - I,maj7`**  
2. Progresión de 4 acordes: **`I - vi - IV - V`**  

Cada una se expandirá en **todas** las octavas, tonalidades (C1..B7) y posibles inversiones, produciendo múltiples archivos `.mid`.


In [2]:
# Ejemplo 1: "ii,7 - V,7 - I,maj7"
progression_1 = "ii,7-V,7-I,maj7"
name_1 = "ii7-V7-Imaj7_demo"

generate_progression(
    progression=progression_1,
    name=name_1,
    output_dir=MIDI_DIR,  # data/midi/
    tempo=DEFAULT_TEMPO
)

# Ejemplo 2: "I - vi - IV - V"
progression_2 = "I-vi-IV-V"
name_2 = "I-vi-IV-V_demo"

generate_progression(
    progression=progression_2,
    name=name_2,
    output_dir=MIDI_DIR,
    tempo=DEFAULT_TEMPO
)

print("¡Progresiones generadas en data/midi!")


Generated progression 'ii,7-V,7-I,maj7' -> folder: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/ii7-V7-Imaj7_demo
Generated progression 'I-vi-IV-V' -> folder: /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi/I-vi-IV-V_demo
¡Progresiones generadas en data/midi!


## 3. Crear archivos .jams extrayendo la tonalidad de cada .mid

### Lógica:
1. Para cada `.mid`, su nombre luce así:
   `C#-3-ii,7-V,7-I,maj7-0.mid`
   - `C#-3` => base note y octava
   - `ii,7-V,7-I,maj7` => la progresión
   - `0` => índice de inversión

2. Parseamos la **base note** (ignorando la octava) para la `key`, y la **progresión** para obtener los acordes en roman numerals, si queremos. 
   - Nota: En nuestro script `generate_progression`, el `progressionName` se pasa como `name`, así que `ii,7-V,7-I,maj7` es `name_1`.

3. Llamamos a `create_jams_file` con:
   - `roman_sequence` => cada token: e.g. `["ii,7", "V,7", "I,maj7"]`
   - `key` => la base note, p. ej. `"C#"`
   - `jam_name` => el archivo base
4. Repetimos para cada `.mid`.


In [4]:
midi_folder_1 = MIDI_DIR / name_1
midi_folder_2 = MIDI_DIR / name_2

In [5]:
def create_jams_for_all_keys(folder: Path):
    """
    Recorre todos los .mid en 'folder', y para cada uno:
    - Parseamos 'C#-3-ii,7-V,7-I,maj7-0.mid'
    - Extraemos la base note (ignorar la octava),
    - Extraemos la progresión,
    - Llamamos create_jams_file
    """
    if not folder.exists():
        print(f"No existe carpeta {folder}")
        return

    mid_files = list(folder.rglob("*.mid"))
    if not mid_files:
        print(f"No hay .mid en {folder}")
        return

    for mf in mid_files:
        base_name = mf.stem  # Sin .mid
        parts = base_name.split("-")
        if len(parts) < 3:
            print(f"Nombre de archivo inesperado: {mf}")
            continue

        # 1) noteName => p.ej. "C#-3" -> parts[0]
        note_plus_oct = parts[0]  # e.g. "C#"
        # Quitar la parte de la octava
        # Ej: "C#-3" => split("-") => ["C#", "3"]
        note_parts = note_plus_oct.split("-")
        if len(note_parts) == 2:
            tonalidad = note_parts[0]  # "C#"
        else:
            tonalidad = note_plus_oct  # fallback

        # 2) Progresión => 'ii,7-V,7-I,maj7' -> parts[1]...parts[-2]
        # En generate_progression, 'name' es 1 token, pero podría contener guiones. 
        # -> Rejuntamos parts[1:-1]
        progression_str = "-".join(parts[1:-1])

        # 3) Indice => last part => parts[-1]
        # lo ignoramos, no es crítico
        # index_str = parts[-1]

        # 4) Roman Sequence => splitted por '-'
        # e.g. "ii,7-V,7-I,maj7" => ["ii,7", "V,7", "I,maj7"]
        roman_sequence = progression_str.split("-")

        # 5) Construimos jam_name = base_name
        #    (para que .jams coincida con .mid)
        jam_name = base_name

        # 6) Llamamos a create_jams_file
        from src.jams_creation import create_jams_file
        jam_path = create_jams_file(
            roman_sequence=roman_sequence,
            key=tonalidad,
            jam_name=jam_name,
            progression_name=progression_str
        )
        print(f"[JAMS] {mf.name} => {jam_path.name}")

# Probamos con midi_folder_1 y midi_folder_2
create_jams_for_all_keys(midi_folder_1)
create_jams_for_all_keys(midi_folder_2)


[JAMS] D-3-ii7-V7-Imaj7_demo-6.mid => D-3-ii7-V7-Imaj7_demo-6.jams
[JAMS] C#-6-ii7-V7-Imaj7_demo-54.mid => C#-6-ii7-V7-Imaj7_demo-54.jams
[JAMS] Bb-7-ii7-V7-Imaj7_demo-35.mid => Bb-7-ii7-V7-Imaj7_demo-35.jams
[JAMS] E-2-ii7-V7-Imaj7_demo-52.mid => E-2-ii7-V7-Imaj7_demo-52.jams
[JAMS] D-8-ii7-V7-Imaj7_demo-59.mid => D-8-ii7-V7-Imaj7_demo-59.jams
[JAMS] G-2-ii7-V7-Imaj7_demo-38.mid => G-2-ii7-V7-Imaj7_demo-38.jams
[JAMS] C-6-ii7-V7-Imaj7_demo-19.mid => C-6-ii7-V7-Imaj7_demo-19.jams
[JAMS] Ab-7-ii7-V7-Imaj7_demo-2.mid => Ab-7-ii7-V7-Imaj7_demo-2.jams
[JAMS] Eb-7-ii7-V7-Imaj7_demo-39.mid => Eb-7-ii7-V7-Imaj7_demo-39.jams
[JAMS] C#-8-ii7-V7-Imaj7_demo-12.mid => C#-8-ii7-V7-Imaj7_demo-12.jams
[JAMS] G-1-ii7-V7-Imaj7_demo-11.mid => G-1-ii7-V7-Imaj7_demo-11.jams
[JAMS] C#-3-ii7-V7-Imaj7_demo-14.mid => C#-3-ii7-V7-Imaj7_demo-14.jams
[JAMS] G-8-ii7-V7-Imaj7_demo-62.mid => G-8-ii7-V7-Imaj7_demo-62.jams
[JAMS] Ab-7-ii7-V7-Imaj7_demo-15.mid => Ab-7-ii7-V7-Imaj7_demo-15.jams
[JAMS] E-4-ii7-V7-Imaj7_

## 4. Convertir las progresiones .mid a .wav con Timidity

Apuntamos a las carpetas generadas en `data/midi/` y convertimos todos los archivos `.mid`.


In [ ]:
print("Carpeta #1:", midi_folder_1)
print("Carpeta #2:", midi_folder_2)

# Conviértelas a WAV
convert_all_mid_in_folder(midi_folder_1)
convert_all_mid_in_folder(midi_folder_2)

print("Archivos WAV generados en data/wav/.")
